In [ ]:
 # import the module
import tweepy
import csv
import time
import tweepy
import pandas as pd
from datetime import datetime
import json
import os
import sys
sys.path.append('drive/gdrive/MyDrive')
from google.colab import drive
drive.mount('/content/gdrive')

#Read file and get only usernames
path="gdrive/MyDrive/AirPorts/"
usernames=pd.read_excel(path+'AirPorts.xlsx')
names=usernames['uname'].values.tolist()
names = [str(x) for x in names]
newlist = [x for x in names if x != 'nan']
unames=[]
for username in newlist:
    unames.append(username.replace("https://twitter.com/", ""))

count=0;
#Authentication
consumer_key        =""
consumer_secret     =""
access_token        =""
access_token_secret =""
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True )
allusersdf = pd.DataFrame(columns=['user','tweets'])
# calling the api
#api = tweepy.API(auth)

api = tweepy.API(auth, parser=tweepy.parsers.JSONParser(),wait_on_rate_limit=True)
for user in unames:
  # screen name of the account to be fetched
  print(user)  
  max_pages=0
  result_limit=0
  count=0
  usertweetcounter=0
  filname= user+ " Tweet"
  
  jsonFile = open(path+filname+'.json', "a+",encoding='utf-8')
  page=1
  max_pages=160
  result_limit=20
  last_tweet_id=False 
  while page <= max_pages:
              if last_tweet_id: 
                  tweet   =   api.user_timeline(screen_name=user,
                                                      count=result_limit,
                                                      max_id=last_tweet_id - 1,
                                                      tweet_mode = 'extended',
                                                      include_retweets=True
                                                      )        
              else:
                  tweet   =   api.user_timeline(screen_name=user,
                                                          count=result_limit,
                                                          tweet_mode = 'extended',
                                                          include_retweets=True)
              if len(tweet)!=0:                 
                json_str = json.dumps(tweet, ensure_ascii=False, indent=4)
                #print(json_str)
                jsonFile.write(json_str)
                usertweetcounter=usertweetcounter+len(tweet)
                print(str(usertweetcounter) + " number of statuses have been fetched for ."+ user)

                for item in tweet:
                  #print(type(item))
                  #print(item)
                  last_tweet_id=item['id']
              else: 
                print("Empty response...")

              #last_tweet_id = item.id
              page += 1
              print('Page ',page,' Last Tweet :',last_tweet_id)
  usertweetcounter=usertweetcounter+len(tweet)
  print(str(usertweetcounter) + " Grand Total for."+ user)
  values_to_add = {'user': user,'tweets':usertweetcounter} #'user','tweets'
  row_to_add = pd.Series(values_to_add)
  allusersdf = allusersdf.append(row_to_add, ignore_index=True)
  print(allusersdf)
  allusersdf.to_csv(path+" The AirPorts Tweet Stats.csv")
allusersdf.to_csv(path+" The AirPorts Tweet Stats.csv")
